# Approximation of the 1-D Wave Equation With PINNs Using PyTorch

This notebook presents an example of approximating the solution to the 1-D wave equation using phyisics informed neural networks. The equation approximated in this notebook is the one-dimensional wave equation, which is a fundamental equation in physics that describes wave propagation. The equation is given by:
$$
\begin{aligned} \partial_t^2 u(x, t)=c^2 \partial_x^2 u(x, t) & \text { for all } 0<x<1 \text { and } t>0 \\ u(0, t)=u(1, t)=0 & \text { for all } t>0, \\ u(x, 0)=x(1-x) & \text { for all } 0<x<1 \\ \partial_t u(x, 0)=0 & \text { for all } 0<x<1\end{aligned}
$$

In this equation, $u(x, t)$ represents the wave function, which gives the position of the wave at location $x$ and time $t$. The term $c^2$ is the square of the wave speed, and $∂_t^2$ and $∂_x^2$ are the second derivatives with respect to time and space, respectively. 


In the following code block, we import the necessary libraries for our notebook. This includes NumPy for numerical operations, PyTorch for building and training the neural network, Matplotlib for plotting, a utility module for additional plotting functions, and the time module for timing our training process.


In [2]:
# Import NumPy for numerical operations
import numpy as np

# Import PyTorch for building and training neural networks
import torch
import torch.nn as nn
import torch.optim as optim

# Import Matplotlib for plotting
import matplotlib.pyplot as plt

# Import a utility module for additional plotting functions
import utils_plots

# Import the time module to time our training process
import time

In the following code block, we define a function for the analytical solution of the 1D wave equation. We then generate training data in NumPy, create a grid of `x` and `t` values, and calculate the corresponding u values. Finally, we plot the u values as a function of `x` and `t`.

The analytical solution to the wave equation is given by the function:

$$
u(t, x) = \sum_{k=1,...,n} \frac{8}{k^3 \pi^3} \sin(k \pi x) \cos(C k \pi t)
$$

In this equation, `C` is a constant set to 1, `k` is an integer that ranges from 1 to `n`, `x` represents the spatial position, and `t` represents time. The sine and cosine functions create a wave pattern in the `x` and `t` dimensions, respectively. The fraction in front of the sine and cosine functions ensures that the wave pattern diminishes for larger values of `k`.

This function is used to generate the `u` values, which represent the wave's displacement at each point in our grid of `x` (spatial position) and `t` (time) values. This grid forms the basis for our training data, which we will use to train our neural network to approximate the wave equation.

In [3]:
# Define a function for the analytical solution of the 1D wave equation
def analytic_sol_func(t, x):
    C = 1
    return sum([(8 / (k**3 * np.pi**3)) * np.sin(k * np.pi * x) * np.cos(C * k * np.pi * t) for k in range(1, 100, 2)])

# Generate training data in NumPy
x_np = np.linspace(0, 1, 100)  # x data (numpy array), shape=(100,)
t_np = np.linspace(0, 1, 100)  # t data (numpy array), shape=(100,)

# Create a grid of x and t values
x_grid, t_grid = np.meshgrid(x_np, t_np) # x and t data (numpy array), shape=(100, 100)

# Calculate u values using the analytical solution function
u_grid = analytic_sol_func(t_grid,x_grid) # u data (numpy array), shape=(100, 100)

In the following code block, we convert our grid data from NumPy arrays to PyTorch tensors, which can be processed more efficiently by our neural network. We also concatenate the spatial (`x`) and temporal (`t`) tensors to form our input data.

In [7]:
# Conversion of the grid data to PyTorch tensors
x = torch.from_numpy(x_grid).float().unsqueeze(-1).requires_grad_(True)
t = torch.from_numpy(t_grid).float().unsqueeze(-1).requires_grad_(True)
u = torch.from_numpy(u_grid).float().unsqueeze(-1)

# Concatenation of x and y to form the input data
input_data = torch.cat((x, t), dim=-1)

In the following code block, we define our neural network model. This model is a simple feed-forward neural network with **two** hidden layers, each containing **50 neurons**. The activation function used is the hyperbolic tangent (`tanh`). We also instantiate the model, and define the optimizer (**Adam**) and the loss function (**Mean Squared Error**) to be used during training.

In [10]:
# Define a neural network class with three fully connected layers
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(2, 50)
        self.layer2 = nn.Linear(50, 50)
        self.output_layer = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.tanh(self.layer1(x))
        x = torch.tanh(self.layer2(x))
        x = self.output_layer(x)
        return x
    
# Create an instance of the neural network
neural_net = NeuralNetwork()

# Define an optimizer (Adam) for training the network
optimizer = optim.Adam(neural_net.parameters(), lr=0.01)

# Define a loss function (Mean Squared Error) for training the network
loss_func = nn.MSELoss()

In the following code block, we train our neural network. We run the training process for 500 iterations, during which we feed the input data to the network, 

$$
\partial_t^2 u(x, t) - c^2 \partial_x^2 u(x, t) = 0 
$$

In [12]:
# Initialize a list to store the loss values
loss_values = []

# Start the timer
start_time = time.time()

# Training the neural network
for i in range(1000):
    prediction = neural_net(input_data)     # input x and predict based on x
    dudx  = torch.autograd.grad(prediction, x, torch.ones_like(prediction), create_graph=True)[0] # computes du/dx
    dudx2 = torch.autograd.grad(dudx,  x, torch.ones_like(dudx),  create_graph=True)[0] # computes d^2u/dx^2
    dudt  = torch.autograd.grad(prediction, t, torch.ones_like(prediction), create_graph=True)[0] # computes du/dt
    dudt2 = torch.autograd.grad(dudt,  t, torch.ones_like(dudt),  create_graph=True)[0] # computes d^2u/dt^2
    loss = dudt2 - dudx2 #loss_func(prediction, u)     # must be (1. nn output, 2. target)
    
    # Append the current loss value to the list
    loss_values.append(loss.item())
    
    if i % 100 == 0:  # print every 100 iterations
        print(f"Iteration {i}: Loss {loss.item()}")
    
    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()

# Stop the timer and calculate the elapsed time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

# Save a summary of the training process to a text file
with open("summaries/summary_1D_Wave_Equation_NN_Model.txt", "w") as file:
    file.write(f"Training time: {elapsed_time} seconds\n")
    file.write(f"Number of iterations: {len(loss_values)}\n")
    file.write(f"Initial loss: {loss_values[0]}\n")
    file.write(f"Final loss: {loss_values[-1]}\n")
    file.write(f"Neural network architecture: {neural_net}\n")
    file.write(f"Optimizer used: {type(optimizer).__name__}\n")
    file.write(f"Learning rate: {optimizer.param_groups[0]['lr']}\n")

Iteration 0: Loss 0.00294599705375731
Iteration 100: Loss 0.00017846233095042408
Iteration 200: Loss 8.040277316467836e-05


KeyboardInterrupt: 